In [5]:
import pandas as pd # 데이터 프레임을 쓰기 위해 사용함

with open("kor_raw.txt", "r", encoding="utf-8") as file:
    lines = file.read()
data = lines.split("$$") # txt 파일에 $$로 행 구분이 되어 있음
df = pd.DataFrame([line.strip().split("||") for line in data[1:-1]], columns=data[0].strip().split("||")) # 마지막 행은 "\n"이라서 추가 안함
df = df.fillna("0") # 결측치를 전부 0으로 대체함
df.head()

,제목,분야,본문,일자,언론사,댓글수,관련뉴스개수
0,"민주당, 권리당원 표 비중 확대‥이재명 ""점진적으로 바꿔나가야""",정치,더불어민주당은 오늘 당무위원회를 열어 당 대표와 최고위원을 뽑는 전당대회에서 권리당...,2023.11.27. 오전 11:32,MBC,19,56
1,"하태경, 종로 출마 선언…""수도권 승리 이끌겠다""",정치,부산 해운대갑에 지역구를 둔 하태경 국민의힘 의원(3선)이 내년 총선에서 서울 종로...,2023.11.27. 오후 6:46,한국경제,1,84
2,"이탄희 ""현 지역구 불출마…기득권 버리고 험지 가겠다""",정치,"지도부에 '연동형 유지·위성정당 방지법' 결단 촉구\n ""두려움 이기고 원칙 지켜야...",2023.11.28. 오전 10:20,아이뉴스24,15,44
3,"여 당무위 ""하위 46개 당협, 공관위 보고…컷오프 비율 22.5%""(1보)",정치,"""현역, 개인 지지도가 정당 지지도보다 현격히 낮은 경우 '문제'""\n [서울=뉴시...",2023.11.27. 오후 6:37,뉴시스,1,79
4,"인요한 혁신위, 안팎 혼란 속 ""준석이 부모 잘못"" 발언 논란",정치,[앵커]\n \n 국민의힘 인요한 혁신위원장이 이준석 전 대표를 향한 발언으로 논란...,2023.11.27. 오후 6:15,연합뉴스TV,8,114


# **<span style="color:blue">데이터 전처리</span>**
---

In [6]:
import re # 뉴스 기사의 특수문자를 제거하기 위해 사용함
from datetime import datetime # 오늘 날짜를 얻기 위해 사용
import numpy as np # 로그 변환을 사용하기 위함
from gensim.summarization.summarizer import summarize # 텍스트 요약 라이브러리
from googletrans import Translator # 번역을 할 수 있는 라이브러리

df["제목"] = df["제목"].apply(lambda x: x.strip().replace("\n"," ").replace("\t"," ")) # 제목 열 특수문자 처리
df["일자"] = df["일자"].apply(lambda x: x.split()[0]) # 일자는 앞쪽에만 있고 뒤에는 시간임
df["일자"] = pd.to_datetime(df["일자"]) # 일자 변수를 데이터트타입 형으로 변경
today = datetime.now().date() # 오늘 날짜
df = df[df['일자'].dt.date == today] # 오늘 뉴스만 가져옴
df = df.reset_index(drop=True) 

df["본문"] = df["본문"].apply(lambda x:re.sub(r'[^\w\s.]', ' ', x).replace('.', '. ').replace('\xa0', '. ').strip()) # 뉴스 본문 특수문자 처리
df = df.astype({"관련뉴스개수": int, "댓글수": int}) # 데이터 타입 지정

df['댓글수_W'] = df.groupby('분야')['댓글수'].transform(lambda x: np.log1p(x)) # 댓글 수 변수는 로그 변환 후 정규화시킴
df['댓글수_W'] = df.groupby('분야')['댓글수_W'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

df['관련뉴스개수_W'] = df.groupby('분야')['관련뉴스개수'].transform(lambda x: (x - x.min()) / (x.max() - x.min())) # 각 분야별로 관련뉴스 개수를 정규화 시킴

w1 = 0.4 # 댓글 수의 가중치
w2 = 0.6 # 관련뉴스 개수의 가중치
df["W"] = df["댓글수_W"] * w1 + df['관련뉴스개수_W']* w2 # 가중합 변수 생성

# 각 카테고리별로 가중합("W변수")이 가장 큰 뉴스를 선정하고, 그것을 "Select" 변수에 1이라는 숫자로 표현함
df['Select'] = (df.groupby('분야')['W'].transform('max') == df['W']).astype(int) 

df["요약"] = df["본문"].apply(lambda x: summarize(x, word_count=50) if isinstance(x, str) else " ") # 요약함

for i in range(len(df["요약"])): # 요약본이 안 생겼을 경우는 본문을 요약 칸에 채움
    if df["요약"][i] == "":
        df["요약"][i] = df["본문"][i]

df["번역"] =" " # 번역 열 생성
def func(x): # 선정된 뉴스의 요약을 번역하는 함수 설정
    translator = Translator() # 번역기 객체 생성
    sentence = x.replace("\n", " ") # 번역을 하기 위해 요약 변수값을 조정
    eng = translator.translate(sentence,  dest = "en") # 번역기 객체를 사용해 한국어 -> 영어 번역
    return eng.text # 번역된 텍스트를 반환

df.loc[df['Select'] == 1, "번역"] = df.loc[df['Select'] == 1, "요약"].apply(func) # 선정된 뉴스의 요약을 번역함

df = df[df["Select"] ==1].reset_index(drop = True) # select가 된 것만 저장

df["이미지"] = " " # 이미지 이름 열 저장 -> DB에 쉽게 저장하기 위함
for i in range(6):
    name = f"kor_image_{i}"
    df["이미지"][i] = name
df = df.drop('본문', axis=1)
df.head()

C:\Users\82106\AppData\Local\Temp\ipykernel_22948\3636572104.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["이미지"][i] = name
C:\Users\82106\AppData\Local\Temp\ipykernel_22948\3636572104.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["이미지"][i] = name
C:\Users\82106\AppData\Local\Temp\ipykernel_22948\3636572104.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["이미지"][i] = name
C:\Users\82106\AppData\Local\Temp\ipy

,제목,분야,일자,언론사,댓글수,관련뉴스개수,댓글수_W,관련뉴스개수_W,W,Select,요약,번역,이미지
0,"이탄희 ""현 지역구 불출마…기득권 버리고 험지 가겠다""",정치,2023-11-28,아이뉴스24,15,44,0.530019,1.0,0.812008,1,현행 선거제도 준연동형 비례대표제 유지와 위성정당 방지법 을 주장하는 이탄희 더...,"Lee Tan -hee, who claims to maintain the curre...",kor_image_0
1,"볼보 EX30, 국내 최초 공개…“전기차 대중화 이끌 것”",경제,2023-11-28,헤럴드경제,1,29,0.630930,1.0,0.852372,1,북유럽 감성 디자인 친환경 기술도 눈길\n인테리어는 중앙집중화 테마와 혁신적인 공간...,In the Nordic Emotional Design Eco -Friendly T...,kor_image_1
2,“그날이 왔다”…부산-파리 이원 생중계 시민응원전 전개,사회,2023-11-28,파이낸셜뉴스,7,47,0.369036,1.0,0.747614,1,박형준 부산시장은 영상메시지를 통해 지난 2년여간 시민들과 함께 해 온 많은 일들에...,Busan Mayor Park Hyung -jun will deliver a mes...,kor_image_2
3,'강풍에 빙판길' 다시 쌀쌀한 출근길…전날보다 3~8도↓[오늘날씨],생활_문화,2023-11-28,뉴스1,1,47,1.000000,1.0,1.000000,1,예상 강수량은 경북내륙 경남서부내륙 울릉도 독도 1 내외 제주도 5 미만이다...,Expected precipitation is less than 5 Jeju -do...,kor_image_3
4,"이·하마스, 휴전 이틀 연장 이어 인질 20명 석방 예정",세계,2023-11-28,헤럴드경제,1,103,0.118855,1.0,0.647542,1,휴전 연장과 함께 지난 4일간 하마스에서 69명의 인질이 석방된 데 이어 이스라엘이...,"Along with the extension of the ceasefire, 69 ...",kor_image_4


# **<span style="color:blue">파일 저장</span>**
---

In [7]:
df.to_csv("kor_scraping.csv", index=True) # csv 저장 => 이미지 저장용

file = open("kor_scraping.txt", "w") # txt 저장 => DB용

for i in range(len(df.to_numpy())):
    file.write(f"{i}||")
    file.write("||".join(map(str,list(df.to_numpy()[i])))) 
    file.write("&&")

file.close()